# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of recommendation is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for recommendation, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)
  
In this hands-on tutorial, we will demonstrate how to write GraphSAGE in DGL + MXNet, and how to apply it in a recommender system setting.

## Dependencies

* Latest DGL release: `conda install -c dglteam dgl`
* `pandas`
* `stanfordnlp`
* `mxnet`
* `tqdm` for displaying the progress bar.

In [ ]:
import mxnet as mx
from mxnet import ndarray as nd, autograd, gluon
from mxnet.gluon import nn
import dgl
import dgl.function as FN
import numpy as np

np.random.seed(0)

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-100K dataset.  The data comes from [MovieLens](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and is shipped with the notebook already.

After loading and train-validation-test-splitting the dataset, we process features into categorical variables (i.e. integers).  We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

All of the above is encapsulated in `movielens.MovieLens` class for clarity of this notebook.

In [ ]:
import movielens

ml = movielens.MovieLens('ml-100k')

In [ ]:
g = ml.g
print('#vertices:', g.number_of_nodes())
print('#edges:', g.number_of_edges())

## See the features in the MovieLens dataset

The MovieLens dataset has some user features and movie features.

User features:
* age,
* gender,
* occupation,
* zip code,

Movie features:
* genre,
* year,

We use one-hot encoding for "age", "gender", "occupation", "zip code" and "year". "genre" uses multi-hop encoding.

In additon, there is a node data "type" that indicates the node type in the bipartite graph. Nodes with "type=1" are user nodes and "type=0" are movie nodes.

User nodes don't have features of movie nodes. These features on the user nodes are filled with 0. Similarly, movie nodes don't have features of user nodes and these features are filled with 0.

In [ ]:
print(g.ndata)

In [ ]:
print('#user nodes:', mx.nd.sum(g.ndata['type'] == 1).asnumpy())
print('#movie nodes:', mx.nd.sum(g.ndata['type'] == 0).asnumpy())

# Sampling in DGL

When the graph scales up, it's impractical to run graph neural networks on the full graph because the node embeddings couldn't fit in the GPU memory.

A natural solution would be partitioning the nodes and computing the embeddings one partition (minibatch) at a time.  The nodes at one convolution layer only depends on their neighbors, rather than all the nodes in the graph, hence reducing the computational cost.  However, if we have multiple layers, and some of the nodes have a lot of neighbors (which is often the case since the degree distribution of many real-world graphs follow [power-law](https://en.wikipedia.org/wiki/Scale-free_network)), computing the embedding of a target node still depends on a large number of nodes in the graph.

Please see our [sampling tutorial](https://doc.dgl.ai/tutorials/models/5_giant_graph/1_sampling_mx.html#sphx-glr-tutorials-models-5-giant-graph-1-sampling-mx-py) for details.

The data and computation dependency of computing the embedding on target node 1 is illustrated in the figure below:

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/Dependency.png" width="400">

*Neighbor sampling* is an answer to further reduce the cost of computing node embeddings.  When aggregating messages, instead of collecting from all neighboring nodes, we only collect from some of the randomly-sampled (for instance, uniform sampling at most K neighbors without replacement) neighbors.

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/neighbor_sampling.png" width="600">

DGL provides `NodeFlow` that stores the computation dependency of nodes in a graph convolutional network. Below shows hwo we can run GraphSage on `NodeFlow`.

# Recommendation model

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/rec_process.png" width="800">

Recommendation with graph neural networks has two steps:
* graph encoder: use graph neural networks to compute node embeddings.
* edge decoder: compute scores on edges with user embeddings and movie embeddings.

The only difference of this class is that it applies on `NodeFlow` instead of `DGLGraph`.

In [ ]:
class GNNRecommender(nn.Block):
    def __init__(self, encoder, decoder):
        super(GNNRecommender, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, nf, users, items):
        h = self.encoder(nf)
        h_users = h[nf.map_from_parent_nid(-1, users, True)]
        h_items = h[nf.map_from_parent_nid(-1, items, True)]
        return self.decoder(h_users, h_items, users, items)

## GraphSage encoder on NodeFlow

The encoder does two things:
* generate the initial user and movie embeddings,
* run GraphSAGE layers on nodes multiple times to compute the final embeddings for rating prediction.

### Algorithm

The algorithm of a single GraphSAGE layer goes as follows for each node $v$:

1. $h_{\mathcal{N}(v)} \gets \mathtt{Sum}_{u \in \mathcal{N}(v)} h_{u}$
2. $h_{v} \gets \sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)}/d_{\mathcal{N}(v)})\right)$
3. $h_{v} \gets h_{v} / \lVert h_{v} \rVert_2$

### Slight modification on the original GraphSage model

In practice, the MovieLens dataset has two types of nodes: users and movies. We need to perform separate node update functions on the two types of nodes.

For the movie nodes,

$h_{m} \gets \sigma\left(W0 \cdot \mathtt{CONCAT}(h_m, h_{\mathcal{N}(m)} / d_{\mathcal{N}(m)})\right)$, 
$h_{m} \gets h_{m} / \lVert h_{m} \rVert_2$

For the user nodes,

$h_{u} \gets \sigma\left(W1 \cdot \mathtt{CONCAT}(h_u, h_{\mathcal{N}(u)} / d_{\mathcal{N}(u)})\right)$,
$h_{u} \gets h_{u} / \lVert h_{u} \rVert_2$

In [ ]:
class GraphSageEncoder(nn.Block):
    def __init__(self, embedding_size, n_layers, G):
        super(GraphSageEncoder, self).__init__()

        self.G = G
        self.user_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1).astype(np.int64)
        self.movie_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0).astype(np.int64)

        self.user_layers = nn.Sequential()
        self.movie_layers = nn.Sequential()
        for i in range(n_layers):
            self.user_layers.add(GraphSageNodeUpdate(embedding_size))
            self.movie_layers.add(GraphSageNodeUpdate(embedding_size))

        # One-hot encoding for each node.
        node_emb = nn.Embedding(G.number_of_nodes() + 1, embedding_size)
        self.user_emb = UserEmbedding(G, embedding_size, node_emb)
        self.movie_emb = MovieEmbedding(G, embedding_size, node_emb)

    def forward(self, nf):
        nf.copy_from_parent(edge_embed_names=None)

        # Generate embeddings on user nodes and movie nodes.
        for i in range(nf.num_layers):            
            layer_nodes = nf.layer_nid(i)
            node_type = nf.layers[i].data['type'].asnumpy()
            user_nodes = layer_nodes[np.nonzero(node_type == 1)]
            movie_nodes = layer_nodes[np.nonzero(node_type == 0)]
            if len(user_nodes) > 0:
                nf.apply_layer(i, lambda nodes: {'h': self.user_emb(nodes.data, user_nodes)},
                               v=user_nodes)
            if len(movie_nodes) > 0:
                nf.apply_layer(i, lambda nodes: {'h': self.movie_emb(nodes.data, movie_nodes)},
                               v=movie_nodes)

        # Apply GraphSage layers on NodeFlow.
        for i in range(nf.num_blocks):
            nf.layers[i+1].data['deg'] = nf.layer_in_degree(i+1).astype(np.float32)
            layer_nodes = nf.layer_nid(i+1).asnumpy()
            node_type = nf.layers[i+1].data['type'].asnumpy()
            user_nodes = layer_nodes[node_type == 1]
            movie_nodes = layer_nodes[node_type == 0]
            nf.block_compute(i, FN.copy_src('h', 'h'), FN.sum('h', 'h_agg'))
            if len(user_nodes) > 0:
                nf.apply_layer(i+1, self.user_layers[i], v=user_nodes)
            if len(movie_nodes) > 0:
                nf.apply_layer(i+1, self.movie_layers[i], v=movie_nodes)

        return nf.layers[-1].data['h']

In [ ]:
class GraphSageNodeUpdate(nn.Block):
    def __init__(self, feature_size):
        super(GraphSageNodeUpdate, self).__init__()

        self.feature_size = feature_size
        self.W = nn.Dense(feature_size)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, nodes):
        # Node embedding from the previous layer.
        h = nodes.data['h']
        # Aggregation of the node embeddings in the neighborhood
        h_agg = nodes.data['h_agg']
        # Degree of the vertex.
        deg = nodes.data['deg'].expand_dims(1)
        h_concat = nd.concat(h, h_agg / nd.maximum(deg, 1e-6), dim=1)
        h_new = self.leaky_relu(self.W(h_concat))
        # Layer norm
        return {'h': h_new / nd.maximum(h_new.norm(axis=1, keepdims=True), 1e-6)}

## Compute node embeddings on the MovieLens dataset

User nodes and movie nodes have different sets of features. Thus, we need to generate embeddings differently.

User nodes have categorial features of "age", "gender", "occupation" and "zip code". These features are all one-hot encodings. In addition, we add one-hot encoding for every user node. To generate user embedding, we add all of these embeddings together.

In [ ]:
class UserEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(UserEmbedding, self).__init__()

        # Embedding matrices for one-hot encoding.
        self.emb_age = nn.Embedding(G.ndata['age'].max().asscalar() + 1,
                                    feature_size)
        self.emb_gender = nn.Embedding(G.ndata['gender'].max().asscalar() + 1,
                                       feature_size)
        self.emb_occupation = nn.Embedding(G.ndata['occupation'].max().asscalar() + 1,
                                           feature_size)
        self.emb_zip = nn.Embedding(G.ndata['zip'].max().asscalar() + 1,
                                    feature_size)

        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_age(ndata['age']))
        extra_repr.append(self.emb_gender(ndata['gender']))
        extra_repr.append(self.emb_occupation(ndata['occupation']))
        extra_repr.append(self.emb_zip(ndata['zip']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

Movie nodes "year", "genre". "year" is one-hot encoding, "genre" is stored in a float32 dense matrix. Like user nodes, we add one-hot encoding to every movie node. To generate movie embedding, we add all of these embeddings together.

In [ ]:
class MovieEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(MovieEmbedding, self).__init__()
        self.emb_year = nn.Embedding(G.ndata['year'].max().asscalar() + 1,
                                     feature_size)

        # Linear projection for float32 features.
        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_genre = seq

        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_year(ndata['year']))
        extra_repr.append(self.proj_genre(ndata['genre']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

## Rating prediction

For recommendation, the rating on item $j$ by user $i$ is defined by $u_i^T v_j$.

In practice, recommendation models have user bias term and movie bias term: $u_i^T v_j + b_{u_i} + b_{v_j}$.

In [ ]:
class DotDecoder(nn.Block):
    def __init__(self, num_nodes):
        super(DotDecoder, self).__init__()
        
        with self.name_scope():
            self.biases = self.params.get(
                'node_biases',
                init=mx.init.Zero(),
                shape=(num_nodes+1,))
            
    def forward(self, h_users, h_items, users, items):
        user_biases = self.biases.data()[users+1]
        item_biases = self.biases.data()[items+1]
        return (h_users * h_items).sum(1) + user_biases + item_biases

## Edge sampling for rating prediction

For rating prediction, we first need to sample a set of edges. On the endpoint nodes of the edges, we run GraphSage to compute their embeddings. Therefore, we sample edges along with `NodeFlow`s. For each batch of sampled edges, we construct a `NodeFlow` for the endpoint of each side. This is illustrated with the figure below:

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/rating_pred.png" width="300">

We can use `NeighborSampler` to implement this edge sampler. When the edge sampler constructs a batch, it creates a `NodeFlow` for the endpoint nodes of both sides as well as endpoint nodes and ratings of the edges.

In [ ]:
batch_size = 1024                       # The number of target nodes in a batch.
num_neighbors = 5                       # The number of sampled neighbors on each node
num_layers = 1.                         # The number of layers in GraphSage.

class EdgeSampler:
    def __init__(self, g, src, dst, rating):
        shuffle_idx = nd.from_numpy(np.random.permutation(g.number_of_edges()))
        src_shuffled = src[shuffle_idx]
        dst_shuffled = dst[shuffle_idx]
        rating_shuffled = rating[shuffle_idx]
        
        self.src_batches = []
        self.dst_batches = []
        self.rating_batches = []
        for i in range(0, g.number_of_edges(), batch_size):
            j = min(i + batch_size, g.number_of_edges())
            self.src_batches.append(src_shuffled[shuffle_idx[i:j]])
            self.dst_batches.append(dst_shuffled[shuffle_idx[i:j]])
            self.rating_batches.append(rating_shuffled[shuffle_idx[i:j]])

        seed_nodes = nd.concat(*sum([[s, d] for s, d in zip(self.src_batches, self.dst_batches)], []), dim=0)
        self.sampler = iter(dgl.contrib.sampling.NeighborSampler(
            g,                     # the graph
            batch_size * 2,        # number of nodes to compute at a time, HACK 2
            num_neighbors,         # number of neighbors for each node
            num_layers,            # number of layers in GCN
            seed_nodes=seed_nodes, # list of seed nodes, HACK 2
            prefetch=True,         # whether to prefetch the NodeFlows
            add_self_loop=True,    # whether to add a self-loop in the NodeFlows, HACK 1
            shuffle=False,         # whether to shuffle the seed nodes.  Should be False here.
            num_workers=4,
        ))
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i == len(self.src_batches):
            raise StopIteration
            
        idx = self.i
        self.i += 1
        return (self.src_batches[idx], self.dst_batches[idx],
                self.rating_batches[idx], next(self.sampler))

## Get the training set

We use 80% of edges for training.

In [ ]:
g_train = ml.g_train
rating_train = g_train.edata['rating']
src_train, dst_train = g_train.all_edges()
print('#vertices:', g_train.number_of_nodes())
print('#training edges:', g_train.number_of_edges())
print('percentage:', g_train.number_of_edges() / g.number_of_edges() * 100, "%")

## Get the testing edge set

We use 20% of edges for evaluation.

In [ ]:
g = ml.g
eid_test = g.filter_edges(lambda edges: edges.data['test']).astype('int64')
src_test, dst_test = g.find_edges(eid_test)
rating_test = g.edges[eid_test].data['rating']
print('#testing edges:', len(eid_test))
print('testing edges:', len(eid_test) / g.number_of_edges() * 100, "%")

## Run the model

The training loop

In [ ]:
def train(g_train, src, dst, batch_size):
    # Training
    tot_loss = 0
    num_batches = 0
    for s, d, r, nodeflow in EdgeSampler(g_train, src, dst, rating_train):
        with mx.autograd.record():
            score = model.forward(nodeflow, s, d)
            loss = ((score - r) ** 2).mean()
            loss.backward()
        trainer.step(1)
        tot_loss += loss.asscalar()
        num_batches += 1
        
    # Return the training loss
    return tot_loss / num_batches

The testing code

In [ ]:
def test(g, batch_size):
    # Validation & Test, we precompute GraphSage output for all nodes first.
    sampler = dgl.contrib.sampling.NeighborSampler(
        g,
        batch_size,
        num_neighbors,
        num_layers,
        seed_nodes=nd.arange(g.number_of_nodes()).astype('int64'),
        prefetch=True,
        add_self_loop=True,
        shuffle=False,
        num_workers=4
    )

    h = []
    for nf in sampler:
        h.append(model.encoder(nf))
    h = nd.concat(*h, dim=0)

    # Compute test RMSE
    score = model.decoder(h[src_test], h[dst_test], src_test, dst_test)
    test_rmse = nd.sqrt(((score - rating_test) ** 2).mean())
    
    return test_rmse.asscalar()

In [ ]:
model = GNNRecommender(GraphSageEncoder(100, 1, g_train),
                       DotDecoder(g_train.number_of_nodes()))
model.collect_params().initialize(ctx=mx.cpu())
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.001, 'wd': 1e-9})

for epoch in range(50):
    loss = train(g_train, src_train, dst_train, batch_size)
    test_rmse = test(g, batch_size)
    print('Training loss:', loss, 'Test RMSE:', test_rmse)